In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matric_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matric_car


In [40]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Features

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0],list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [42]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x ]
cat_feats = [x for x in cat_feats if 'price' not in x ]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)


##DecisionTree

In [44]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015937)

## Random Forest

In [45]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost

In [46]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}
 
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[20:04:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:05:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:05:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [55]:
m = xgb.XGBRegressor(**xgb_params)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(m, feature_names=cat_feats)

[20:15:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: ignored

In [49]:
feats = ['param_faktura-vat__cat', 'param_napęd__cat', 'seller_type__cat', 'feature_kamera-cofania__cat', 'param_skrzynia-biegów__cat', 'param_rok-produkcji__cat', 'param_pojemność-skokowa__cat', 'param_vat-marża__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_wspomaganie-kierownicy__cat', 'param_moc__cat', 'param_stan__cat', 'feature_światła-led__cat', 'feature_bluetooth__cat', 'feature_klimatyzacja-czterostrefowa__cat', 'feature_światła-do-jazdy-dziennej__cat', 'feature_czujnik-martwego-pola__cat', 'feature_regulowane-zawieszenie__cat', 'feature_system-start-stop__cat', 'seller_name__cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:12:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-14390.603102888097, 77.07003044844856)

In [50]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = ['param_faktura-vat__cat', 'param_napęd__cat', 'seller_type__cat', 'feature_kamera-cofania__cat', 'param_skrzynia-biegów__cat', 'param_rok-produkcji', 'param_pojemność-skokowa__cat', 'param_vat-marża__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_wspomaganie-kierownicy__cat', 'param_moc__cat', 'param_stan__cat', 'feature_światła-led__cat', 'feature_bluetooth__cat', 'feature_klimatyzacja-czterostrefowa__cat', 'feature_światła-do-jazdy-dziennej__cat', 'feature_czujnik-martwego-pola__cat', 'feature_regulowane-zawieszenie__cat', 'feature_system-start-stop__cat', 'seller_name__cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:12:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-12832.547385886324, 82.52326475533782)

In [52]:
#df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
#df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = ['param_faktura-vat__cat', 'param_napęd__cat', 'seller_type__cat', 'feature_kamera-cofania__cat', 'param_skrzynia-biegów__cat', 'param_rok-produkcji', 'param_pojemność-skokowa__cat', 'param_vat-marża__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_wspomaganie-kierownicy__cat', 'param_moc', 'param_stan__cat', 'feature_światła-led__cat', 'feature_bluetooth__cat', 'feature_klimatyzacja-czterostrefowa__cat', 'feature_światła-do-jazdy-dziennej__cat', 'feature_czujnik-martwego-pola__cat', 'feature_regulowane-zawieszenie__cat', 'feature_system-start-stop__cat', 'seller_name__cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:13:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:13:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:13:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10780.535345974415, 84.26045671819098)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')))

In [54]:
feats = ['param_faktura-vat__cat', 'param_napęd__cat', 'seller_type__cat', 'feature_kamera-cofania__cat', 'param_skrzynia-biegów__cat', 'param_rok-produkcji', 'param_pojemność-skokowa', 'param_vat-marża__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_wspomaganie-kierownicy__cat', 'param_moc', 'param_stan__cat', 'feature_światła-led__cat', 'feature_bluetooth__cat', 'feature_klimatyzacja-czterostrefowa__cat', 'feature_światła-do-jazdy-dziennej__cat', 'feature_czujnik-martwego-pola__cat', 'feature_regulowane-zawieszenie__cat', 'feature_system-start-stop__cat', 'seller_name__cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:14:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:14:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:14:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10553.109700067094, 68.35948850620984)